In [ ]:
import h5py
import matplotlib.pyplot as plt
import mpl_lego.colors as colors
import neuropacks as packs
import numpy as np
import os

from mpl_lego.ellipse import plot_cov_ellipse
from neurobiases import analysis
from neurobiases.utils import all_correlations

from mpl_lego.labels import bold_text

%matplotlib inline

In [ ]:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.sans-serif": ["Computer Modern Roman"]})

In [ ]:
ccycle = colors.get_default_ccycle()

In [ ]:
# Base paths
base_path = '/storage/data'
# PVC11 path
pvc11_path = os.path.join(base_path, 'pvc11/data/spikes_gratings/data_monkey1_gratings.mat')
ecog_path = os.path.join(base_path, 'ecog/r32_b7.mat')
# Create neuropack
pvc11 = packs.PVC11(data_path=pvc11_path)
ecog = packs.ECOG(data_path=ecog_path)

In [ ]:
hg_responses = np.mean(ecog.responses[:, :, :, :, :, 2], axis=1).reshape((101, -1)).T

In [ ]:
ecog_highest = np.flip(np.argsort(np.max(hg_responses, axis=1)))

In [ ]:
pvc11 = packs.PVC11(pvc11_path)
spike_times = pvc11.events.ravel()

In [ ]:
counts = np.zeros_like(spike_times)

for neuron in range(counts.size):
    counts[neuron] = spike_times[neuron].size

pvc_highest = np.flip(np.argsort(counts))

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
plt.subplots_adjust(wspace=0.4, hspace=0.5)

for idx, neuron_idx in enumerate(pvc_highest[:20]):
    times = spike_times[neuron_idx]
    axes[0, 0].scatter(1000 * times, idx * np.ones_like(times), marker='|', color='black', s=100)

for idx, e_idx in enumerate(ecog_highest[0:10000:500]):
    ecog_response = hg_responses[e_idx]
    ecog_response /= np.max(ecog_response)
    times = np.arange(ecog_response.size) * 1000 / 200
    axes[1, 0].plot(times, 1.2 * idx + ecog_response, color='black')
    

X_pvc11 = pvc11.get_response_matrix()
stimuli_pvc11 = pvc11.get_design_matrix()
ncs_pvc11 = all_correlations(X_pvc11, stimuli_pvc11)

axes[0, 1].hist(ncs_pvc11, histtype='step', color='black', lw=2)

X_ecog = ecog.get_response_matrix(bounds=(40, 60), band='HG')
stimuli_ecog = ecog.get_design_matrix(form='frequency')
ncs_ecog = all_correlations(X_ecog, stimuli_ecog)

axes[1, 1].hist(ncs_ecog, histtype='step', color='black', lw=2)



axes[0, 1].hist(ncs_pvc11, histtype='step', color='black', lw=2)
axes[0, 0].set_xlim([0, 400])
axes[1, 0].set_xlim([0, 400])

axes[0, 1].set_xlim([-1, 1])
axes[1, 1].set_xlim([-1, 1])

axes[0, 0].set_xticks([0, 100, 200, 300, 400])
axes[0, 1].set_xticks([1, -0.5, 0, 0.5, 1.])
axes[1, 0].set_xticks([0, 100, 200, 300, 400])
axes[1, 1].set_xticks([-1, -0.5, 0, 0.5, 1])

axes[0, 0].set_xlabel(bold_text('Time since onset (ms)'), fontsize=15)
axes[1, 0].set_xlabel(bold_text('Time since onset (ms)'), fontsize=15)
axes[0, 1].set_xlabel(bold_text('Noise Correlation'), fontsize=15)
axes[1, 1].set_xlabel(bold_text('Noise Correlation'), fontsize=15)

axes[0, 0].set_ylabel(bold_text('Unit'), fontsize=15)
axes[1, 0].set_ylabel(bold_text('Unit'), fontsize=15)
axes[1, 1].set_ylabel(bold_text('Frequency'), fontsize=15)
axes[0, 1].set_ylabel(bold_text('Frequency'), fontsize=15)


for ax in axes[:, 0]:
    ax.set_yticks([])
    
for ax in axes.ravel():
    ax.tick_params(labelsize=13)
    
plt.savefig('figure1_sub.pdf', bbox_inches='tight')